<a href="https://colab.research.google.com/github/v4ndi/DataStructuresImplementation/blob/master/Create_streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 1.5 MB/s eta 0:00:00


In [2]:
%%writefile app.py

import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Инициализация модели и токенайзера
tokenizer = AutoTokenizer.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("ai-forever/rugpt3large_based_on_gpt2")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Функция генерации текста
def generate_text(prompt, max_new_tokens, min_new_tokens):
    encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
    output = model.generate(
        **encoded_input,
        num_beams=6,
        do_sample=True,
        max_new_tokens=max_new_tokens,
        min_new_tokens=min_new_tokens,
        temperature=1.5,
    )
    return tokenizer.decode(output[0], skip_special_tokens=True), len(output[0])


# Вычисление перплексии
def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss.item()
    perplexity = torch.exp(torch.tensor(loss))
    return perplexity.item()

# Вычисление вероятностной оценки повторений
def calculate_repetition_rate(text):
    vectorizer = CountVectorizer()
    word_counts = vectorizer.fit_transform([text]).toarray()
    unique_words = np.count_nonzero(word_counts)
    total_words = sum(word_counts[0])
    return 1 - unique_words / total_words


# Streamlit интерфейс
st.title("Генерация текста с оценкой качества")

# Ввод для генерации текста
prompt = st.text_area("Введите текст запроса", "Напиши аннотацию к научной работе на тему...")
max_new_tokens = st.slider("Максимальное количество новых токенов", 50, 500, 150)
min_new_tokens = st.slider("Минимальное количество новых токенов", 10, 100, 10)

if st.button("Сгенерировать текст"):
    generated_text, _ = generate_text(prompt, max_new_tokens, min_new_tokens)
    perplexity = calculate_perplexity(generated_text)
    repetition_rate = calculate_repetition_rate(generated_text)

    # Отображение сгенерированного текста и показателей
    st.subheader("Сгенерированный текст")
    st.write(generated_text)

    st.subheader("Оценки качества текста")
    st.write(f"Перплексия: {perplexity:.2f}")
    st.write(f"Уровень повторов: {repetition_rate:.2f}")


# Ввод для продолжения текста
continuation_prompt = st.text_area("Введите текст для продолжения", "Концепция качества исходного кода...")
if st.button("Сгенерировать продолжение"):
    min_len_paragraph = len(generated_text) * 2 if len(generated_text) * 2 < 330 else 330
    generated_paragraph, _ = generate_text(continuation_prompt, 330, min_len_paragraph)
    perplexity = calculate_perplexity(generated_paragraph)
    repetition_rate = calculate_repetition_rate(generated_paragraph)

    # Отображение сгенерированного параграфа и показателей
    st.subheader("Сгенерированный параграф")
    st.write(generated_paragraph)

    st.subheader("Оценки качества параграфа")
    st.write(f"Перплексия: {perplexity:.2f}")
    st.write(f"Уровень повторов: {repetition_rate:.2f}")

Writing app.py


In [3]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [4]:
!streamlit run /content/app.py &>/content/logs.txt &

In [5]:
!npx localtunnel --port 8501

your url is: https://chilly-rocks-fly.loca.lt
^C


[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)